<a href="https://colab.research.google.com/github/Emmanuel1989/DE-101-Practices/blob/master/sub_chem_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import xml.etree.ElementTree as ET
mytree = ET.parse('part_12_complete.xml')
root = mytree.getroot()
for child in root:
    print(child.tag)

{http://www.org.cas.cds/CMS}CMS_DBP_parent_uri
{http://www.org.cas.cds/CMS}CMSdoc
{http://www.org.cas.cds/CMS}contacts_grp
{http://www.org.cas.cds/CMS}chemcats_includes
{http://www.org.cas.cds/CMS}vendor_specific


In [114]:
from dataclasses import dataclass

@dataclass
class Substance:
  experimental: str
  url: str
  substance_category: str
  substance_full_name: str
  bulk: str
  screening: str
  order_no: str

In [120]:
elem1 = root.find("{http://www.org.cas.cds/CMS}CMSdoc").find("{http://www.org.cas.cds/CMS}doc_type_grp").find("{http://www.org.cas.cds/CMS}doc_type_specific_catalog").find("{http://www.org.cas.cds/CMS}catalog_entries")
elem2 = elem1.findall("{http://www.org.cas.cds/CMS}catalog_entry")
list_substance_category = []
for elem in elem2:
  sub_full_name = ""
  sub_category = ""
  exp_props = elem.find("{http://www.org.cas.cds/CMS}experimental_props").text
  url = elem.find("{http://www.org.cas.cds/CMS}url").text
  order_no = elem.find("{http://www.org.cas.cds/CMS}order_number").text
  if elem.find("{http://www.org.cas.cds/CMS}substance_category") != None:
    sub_category = elem.find("{http://www.org.cas.cds/CMS}substance_category").text
  if elem.find("{http://www.org.cas.cds/CMS}catalog_substance_name") != None and elem.find("{http://www.org.cas.cds/CMS}catalog_substance_name").find("{http://www.org.cas.cds/CMS}subst_name_full") != None:
    sub_full_name = elem.find("{http://www.org.cas.cds/CMS}catalog_substance_name").find("{http://www.org.cas.cds/CMS}subst_name_full").text
  bulk =elem.find("{http://www.org.cas.cds/CMS}bulk").text
  screening = elem.find("{http://www.org.cas.cds/CMS}screening").text
  sub = Substance(experimental= exp_props, url=url, substance_category=sub_category, substance_full_name=sub_full_name, bulk=bulk, screening=screening, order_no=order_no)
  list_substance_category.append(sub)

In [91]:
from dataclasses import dataclass

@dataclass
class catalog_items:
    chemical_name: str
    substance: str
    order_no: str
    purity_category: str
    molar_weight: float
    is_hazzard: bool

In [76]:
print(list_substance_category[1])

Substance(experimental='No', url='http://www.carbosynth.com/sf/FD95815', substance_category='Supplied by supplier', substance_full_name='3-(Difluoromethyl)-1,4,5,6-tetrahydrocyclopenta[c]pyrazole', bulk='No', screening='No', order_no='FD95815')


In [104]:
elem3 = root.find("{http://www.org.cas.cds/CMS}vendor_specific").find("catalog-publishable").find("catalog-items").findall("catalog-item")
list_catalog_items = []
for elem in elem3:
  mw = ""
  is_hazzard = False
  chem_name = ""
  purity_cat = ""
  if elem.find("chemical-name") != None:
    chem_name = elem.find("chemical-name").text
  if elem.find("substance") != None:
    substance = elem.find("substance").text
  order_no = elem.find("order-number").text
  if elem.find('purity-category') != None:
    purity_cat = elem.find('purity-category').text
  if elem.find("supplier-reported-properties") != None:
    if elem.find("supplier-reported-properties").findall("supplier-reported-property") != None:
      for e in elem.find("supplier-reported-properties").findall("supplier-reported-property"):
        if e.find("context").text == "MW":
          ##Found molar weigth
          mw = e.find("value").text
        if e.find("context").text == "hazzard":
          ##Is hazzard found
          is_hazzard = e.find("value").text
  cat_item = catalog_items(chemical_name=chem_name, substance=substance, order_no=order_no, purity_category=purity_cat,molar_weight=mw ,is_hazzard=is_hazzard)
  list_catalog_items.append(cat_item)

print(list_catalog_items[0])

catalog_items(chemical_name='3-(Difluoromethyl)-1,4,5,6-tetrahydrocyclopenta[c]pyrazole', substance='938022-30-1', order_no='FD95815', purity_category='', molar_weight='158.15', is_hazzard=False)


In [105]:
##Convert to dataframe  list catalog items
from dataclasses import dataclass, asdict
import pandas as pd
df_cat_items = pd.DataFrame.from_records([asdict(s) for s in list_catalog_items])

In [121]:
##Convert to dataframe  list substances items
from dataclasses import dataclass, asdict
import pandas as pd
df_substance_items = pd.DataFrame.from_records([asdict(s) for s in list_substance_category])

In [108]:
df_cat_items.head(n=20)

,chemical_name,substance,order_no,purity_category,molar_weight,is_hazzard
0,"3-(Difluoromethyl)-1,4,5,6-tetrahydrocyclopent...",938022-30-1,FD95815,,158.15,False
1,"{2-[(9,9-DiMethyl-9H-fluoren-2-yl)-diMethyl-si...",1244855-65-9,FD40079,,,False
2,"Thiomorpholine-1,1-dioxide HCl",59801-62-6,FT38522,,,False
3,Phenylisothiocyanate,103-72-0,FP01158,Extremely High,135.19,False
4,"4-(2,4-Dimethylphenyl)-1,3-thiazol-2-amine",247225-31-6,FD113273,,204.29,False
5,2-Carboethoxybenzene sulfonamide,59777-72-9,FC38144,,,False
6,3-Fluoropentane,41909-29-9,FF77108,,90.14,False
7,Methyl 4-benzyloxy-3-methoxybenzoate,56441-97-5,FM32989,,272.30,False
8,Methyl (E)-3-(3-bromophenyl)-2-propenoate,79432-87-4,FM43103,,,False
9,Zirconiumdicarbonate,36577-48-7,FZ150107,,211.24,False


In [123]:
df_substance_items.drop_duplicates(keep=False, inplace=True)

<bound method IndexOpsMixin.value_counts of 0        FD95815
1        FD40079
2        FT38522
3        FP01158
4       FD113273
          ...   
5672     FB88848
5673    FA100954
5674     FB80530
5675    FB140901
5676    FM125670
Name: order_no, Length: 5677, dtype: object>

In [125]:
df_complete = df_substance_items.set_index('order_no').join(df_cat_items.set_index('order_no'))

In [126]:
df_complete.head(n=20)

,experimental,url,substance_category,substance_full_name,bulk,screening,chemical_name,substance,purity_category,molar_weight,is_hazzard
order_no,,,,,,,,,,,
FD95815,No,http://www.carbosynth.com/sf/FD95815,Supplied by supplier,"3-(Difluoromethyl)-1,4,5,6-tetrahydrocyclopent...",No,No,"3-(Difluoromethyl)-1,4,5,6-tetrahydrocyclopent...",938022-30-1,,158.15,False
FD40079,No,http://www.carbosynth.com/sf/FD40079,Supplied by supplier,"{2-[(9,9-DiMethyl-9H-fluoren-2-yl)-diMethyl-si...",No,No,"{2-[(9,9-DiMethyl-9H-fluoren-2-yl)-diMethyl-si...",1244855-65-9,,,False
FT38522,No,http://www.carbosynth.com/sf/FT38522,Supplied by supplier,"Thiomorpholine-1,1-dioxide HCl",No,No,"Thiomorpholine-1,1-dioxide HCl",59801-62-6,,,False
FP01158,No,http://www.carbosynth.com/sf/FP01158,Supplied by supplier,Phenylisothiocyanate,No,No,Phenylisothiocyanate,103-72-0,Extremely High,135.19,False
FD113273,No,http://www.carbosynth.com/sf/FD113273,Supplied by supplier,"4-(2,4-Dimethylphenyl)-1,3-thiazol-2-amine",No,No,"4-(2,4-Dimethylphenyl)-1,3-thiazol-2-amine",247225-31-6,,204.29,False
FC38144,No,http://www.carbosynth.com/sf/FC38144,Supplied by supplier,2-Carboethoxybenzene sulfonamide,No,No,2-Carboethoxybenzene sulfonamide,59777-72-9,,,False
FF77108,No,http://www.carbosynth.com/sf/FF77108,Supplied by supplier,3-Fluoropentane,No,No,3-Fluoropentane,41909-29-9,,90.14,False
FM32989,No,http://www.carbosynth.com/sf/FM32989,Supplied by supplier,Methyl 4-benzyloxy-3-methoxybenzoate,No,No,Methyl 4-benzyloxy-3-methoxybenzoate,56441-97-5,,272.30,False
FM43103,No,http://www.carbosynth.com/sf/FM43103,Supplied by supplier,Methyl (E)-3-(3-bromophenyl)-2-propenoate,No,No,Methyl (E)-3-(3-bromophenyl)-2-propenoate,79432-87-4,,,False
